# Linear Algebra

This will cover some of the maths described in the course, and show you how you can implement it in python.

First, let's import the libraries we'll be using in this notebook:

In [12]:
import numpy as np
import matplotlib.pyplot as plt

As we covered briefly in the numpy notebook, vectors and matrices are represented in numpy as arrays of the right dimensions.

In [13]:
vector = np.array([1,2,3])
matrix = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

You could implement the dot and cross products yourself in code:

In [14]:
def my_dot_product(a, b):
    return np.sum(a * b)

def my_cross_product(a, b):
    return np.array([a[1]*b[2] - a[2]*b[1], a[2]*b[0] - a[0]*b[2], a[0]*b[1] - a[1]*b[0]])

# The dot product of a vector with itself is its length squared:
print(my_dot_product(vector, vector))

# The cross product of any vector with itself is always the zero vector:
print(my_cross_product(vector, vector))

14
[0 0 0]


But thankfully numpy has functions implementing these already:

In [15]:
print(np.dot(vector, vector))
print(np.cross(vector, vector))

14
[0 0 0]


TODO visualise some cross products here using matplotlib's 3D plotting

As a reminder, all the standard mathematical operators in numpy `+ - * /` work **element-wise**. This means that `+` is vector addition, but `*` is **not** matrix multiplication.

To do matrix multiplication you must use the np.matmul() function.

In [16]:
another_vector = np.array([4, 5, 6])
vector_sum = vector + another_vector # This works fine, and does vector addition as you would expect
print(vector_sum)

multiple = matrix * vector # This does not do matrix multiplication
# This will use numpy's "broadcasting" feature, and make copies of the vector to make it a 3x3 matrix.
# Then it will multiply both together, giving a 3x3 result
print(multiple)

# To do real matrix multiplication, use np.matmul()
matrix_multiply = np.matmul(matrix, vector)
print(matrix_multiply)

[5 7 9]
[[1 0 0]
 [0 2 0]
 [0 0 3]]
[1 2 3]


Numpy can also invert a matrix for you:

In [17]:
matrix = np.random.rand(3,3)

matrix_inverse = np.linalg.inv(matrix)

# When multiplying the matrix by its inverse, the result should be close to the identity matrix
# It won't be exactly the same due to floating point errors
multiple = np.matmul(matrix_inverse, matrix)
print(multiple)

# Because it's not exactly the identity, this test won't work:
print(np.all(multiple == np.eye(3)))

# If you want to check if one numpy array is "close enough" to another, you can use np.allclose():
print(np.allclose(np.matmul(matrix, matrix_inverse), np.eye(3)))


[[ 1.00000000e+00  0.00000000e+00  1.11022302e-16]
 [ 2.22044605e-16  1.00000000e+00 -2.22044605e-16]
 [-1.66533454e-16  0.00000000e+00  1.00000000e+00]]
False
True


You can only invert square matrices, so the following won't work:

In [18]:
matrix = np.random.rand(2,3)
matrix_inverse = np.linalg.inv(matrix) # This throws LinAlgError as the input isn't square.

LinAlgError: Last 2 dimensions of the array must be square

As we covered in the course, TODO talk about SVD